# Specification of a model 

##  A Model  

For the purpose of this note, a (simple) model can be thought of as a system of equations that links endogenous and exogenous variables over time.  

 - **n**: number of endogenous variables  
 - **k**: number of exogenous variables  
 - **r**: maximum lag of endogenous variables  
 - **s**: maximum lag of exogenous variables  
 - **t**: time index (year, quarter, month, day, or any other unit)  


\begin{aligned}
y_t^1  &= \;f^1(\; y_t^2,\ldots,y_t^n \;\;,\;\;y_{t-1}^1,\ldots,y_{t-r}^n\;\;,\;\;x_t^1,\ldots,x_t^k\;\;,\;\;x_{t-1}^1,\ldots,x_{t-s}^k \;) \\
y_t^2  &= \;f^2(\; y_t^1,\ldots,y_t^n \;\;,\;\;y_{t-1}^1,\ldots,y_{t-r}^n\;\;,\;\;  x_t^1,\ldots,x_t^k\;\;,\;\; x_{t-1}^1,\ldots,x_{t-s}^k \;) \\
       & \;\;\vdots \\
y_t^n  &= f^n\bigl(\; 
   \underbrace{\underbrace{y_t^1,\ldots,y_t^{n-1}}_{\text{current}} \;,\;
               \underbrace{y_{t-1}^1,\ldots,y_{t-r}^n}_{\text{lagged}}}_{\text{endogenous}},\;
   \underbrace{\underbrace{x_t^1,\ldots,x_t^k}_{\text{current}} \;,\;
               \underbrace{x_{t-1}^1,\ldots,x_{t-s}^k}_{\text{lagged}}}_{\text{exogenous}}
   \;\bigr)
\end{aligned}


Sometime such models are implemented directly in a programming environment such as Python, Matlab, or even Excel, where they are both *specified* and *solved* in the same code.  

---

## Why Separate Specification and Implementation?  

Instead of mixing economic reasoning with programming details, a model can be written in a **domain-specific language (DSL)** designed specifically for economics and finance. Separating specification from implementation has several advantages:  

1. **Clarity and focus** – The specification remains close to the underlying economic logic, expressed in a concise and readable form. The modeler concentrates on *what the relationships are*, not on *how the computer solves them*.  

2. **Expressiveness** – The DSL uses syntax familiar to economists (equations, lags, variables), avoiding the clutter of general-purpose programming constructs.  

3. **Maintainability** – Models can be extended or updated without rewriting solution code. Changing an equation is as simple as editing one line in the specification.  

4. **Portability** – Once specified, the same model can be executed with different numerical engines or platforms without altering its economic content.  

5. **Transparency** – The separation makes models easier to review, validate, and communicate, since the specification reads like economic theory rather than computer code.  

This approach is common in several econometric systems such as **EViews**, **Dynare**, and **GEKKO**, which all use their own DSLs to describe models.  

**ModelFlow** follows a similar philosophy but with a key difference: it does not introduce a new language for data handling, visualization, or model analysis. Instead, ModelFlow relies on the **Python ecosystem** - primarily `pandas`, but also `networkx`, `sympy`, `numpy`, and other libraries - allowing the model specification to be combined seamlessly with powerful tools for data manipulation, result analysis, and visualization.  


## A Simple Specification Language  

In practice, the DSL for economic models is built around equation-like statements. For example:  
```
Frml GDP = C + I + G + X - M $
Frml <stoc> C   = a0 + a1*Yd(-1) $
```

Here each formula (`Frml`) defines the relationship between variables in a way that mirrors how economists usually write them, with lags (`(-1)`) and simple algebraic notation.  

The optional `<tag>` is used for transforming the frml or to control solving - more on that later.

This kind of business-logic-oriented specification acts as the “blueprint” of the model, leaving the task of solving it to the underlying implementation engine.  


## Lists 
In order to facilitate the specification of models with many formulas that follow the same structural template, the DSL also supports **lists** and **sublists**.  

Lists allow the modeler to define groups of entities (such as banks, portfolios, sectors, or countries) and to attach attributes to them. Sublists make it possible to organize these groups into meaningful subcategories, such as assigning each bank to a home country or each portfolio to a regulatory category.  

This mechanism greatly reduces repetition: instead of writing dozens or even hundreds of nearly identical equations, the modeler can write a single template equation that is automatically expanded over the members of a list.  

Such functionality is especially valuable in:  

- **Stress-testing models** – where many banks, each with multiple portfolios, must be represented consistently.  
- **Macroeconomic models** – where different sectors of the economy (households, firms, government, rest of the world, etc.) share the same behavioral structure.  
- **Cross-country or panel models** – where the same specification is applied to multiple countries or regions.  

By using lists and sublists, the specification remains compact, readable, and scalable, while the underlying model can grow to cover large and complex systems.  

List specification in the basic DSL follows these rules: 

- **always starts** with the keyword `list`  
- **ends** with a `$`  
- can contain one or more **sublists**, separated by `/`  
- each sublist starts with a `name:` followed by its members  

Some examples 

```
list BANKS = BANKS    : Danske  Nordea Ibs /
             bankcountry : denmark sweden denmark  $

list country = country : denmark sweden $

list ports = ports  : households   NFC    RE  sov /
               reg  : AIRB  AIRB   STA  STA $

list country__danske = loancountry : uk  DK IR $
list country__nordea = loancountry : SE  DK $
list country__ibs    = loancountry : DK ir $
```
So the list statement starts with a list and ends with a $ sublists are seperated by / and each starts with a `sublist name:` 

Meaning


- `list BANKS = ...` defines a top-level list of banks with an **sublist** `bankcountry` that specifies where each bank belongs.  
- `list country = ...` creates a country list.  
- `list ports = ...` groups portfolios (`households`, `NFC`, `RE`, `sov`) and creates **sublists** with regulatory categories (`AIRB`, `STA`).  
- `list country__danske`, `list country__nordea`, and `list country__ibs` are **lists**, linking each bank to its loan countries.  

The benefits of lists: 


- **Compactness** – groups can be declared once and reused throughout the model.  
- **Flexibility** – adding or removing members requires only updating the list, not the equations.  
- **Transparency** – clear structure makes it easy to see which entities and attributes the model covers.  
